In [ ]:
# !pip install category_encoders 
# !pip install optuna 
import pandas as pd
import numpy as np
import random 
import re 
import os 
import warnings
warnings.filterwarnings("ignore")
import category_encoders as ce 

from sklearn.metrics import (    accuracy_score,    confusion_matrix,f1_score,precision_score, recall_score,roc_auc_score)
from sklearn.metrics import make_scorer
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold,cross_val_score, StratifiedShuffleSplit

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE , ADASYN

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import BaggingClassifier,RandomForestClassifier,ExtraTreesClassifier,GradientBoostingClassifier,HistGradientBoostingClassifier,AdaBoostClassifier
from sklearn.ensemble import VotingClassifier,StackingClassifier
from sklearn.neighbors import KNeighborsClassifier
from catboost import CatBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

import optuna
from optuna.samplers import TPESampler

from sklearn.base import BaseEstimator, TransformerMixin, ClassifierMixin, clone
from sklearn.preprocessing import FunctionTransformer, StandardScaler, LabelEncoder

In [ ]:
pd.set_option('display.max_rows', 500)
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42)
nation_corp = {
    'Austria': ['LGEAG'],    'Czech Republic': ['LGECZ'],    'France': ['LGEFS'],    'Germany': ['LGEDG'],    'Greece': ['LGEHS'],    'Hungary': ['LGEMK'],    'Italy': ['LGEIS'],    'Netherlands': ['LGESC', 'LGEEH', 'LGEBN'],    'Poland': ['LGEWR', 'LGEPL', 'LGEMA'],    'Portugal': ['LGEPT','LGEBT'],
    'EUs': ['LGEEB'],    'Romania': ['LGERO'],    'Spain': ['LGEES'],    'Sweden': ['LGENO', 'LGESW'],    'United Kingdom': ['LGEUK'],      'Kazakhstan': ['LGEAK'],    'Russia': ['LGERM', 'LGERI', 'LGERA'],
    'Ukraine': ['LGEUR'],    'Latvia': ['LGELV','LGELA'],    'Algeria': ['LGEAS'],
    'Egypt': ['LGEEG'],    'Jordan': ['LGELF'],    'Kenya': ['LGESK','LGEEF'],    'Morocco': ['LGEMC'],
    'Saudi Arabia': ['LGESJ'],    'Iran':['LGEIR'],     'Israel':['LGEYK'],     'The Republic of South Africa': ['LGESA'],
    'Tunisia': ['LGETU'],    'U.A.E': ['LGEOT', 'LGEDF', 'LGEGF', 'LGEME', 'LGEAF'],    'Nigeria': ['LGEAO', 'LGENI'],
    'Turkey': ['LGETK', 'LGEAT'],    'Australia': ['LGEAP'],
    'China': ['LGEQA', 'LGETL', 'LGECH', 'LGEYT', 'LGETR', 'LGETA', 'LGESY', 'LGESH', 'LGEQH', 'LGEQD', 'LGEPN', 'LGEND', 'LGEKS', 'LGEHZ', 'LGEHN', 'LGEHK'],
    'India': ['LGEIL'],    'Indonesia': ['LGEIN'],    'Japan': ['LGEJP'],    'Malaysia': ['LGEML'],    'Philippines': ['LGEPH'],
    'Singapore': ['LGESL'],    'Taiwan': ['LGETT'],    'Korea' :['LGEKR'],    'Thailand': ['LGETH'],    'Vietnam': ['LGEVN','LGEVH'],
     'Canada': ['LGECI'],    'Mexico': ['LGERS', 'LGEMX', 'LGEMS', 'LGEMM'],    'United States': ['LGEMR', 'LGEUS', 'LGEMU', 'LGEAI'],
    'Argentina': ['LGEAG','LGEAR'],    'Brazil': ['LGEBR','LGESP'],    'Chile': ['LGECL'],    'Colombia': ['LGEVZ', 'LGECB'],
    'Panama': ['Guatemala', 'LGEPS'],    'Peru': ['LGEPR']}
continent_nation={
    'Europe':['EUs','Austria', 'Czech Republic' ,'France' ,'Germany', 'Greece' ,'Hungary', 'Italy', 'Netherlands' ,'Poland' ,'Portugal' ,'Romania', 'Spain' ,'Sweden','United Kingdom'], 
    'Russia and CIS':['Kazakhstan','Russia', 'Ukraine', 'Latvia'],     'Africa and MiddleEast': ['Israel','Iran','Algeria', 'Egypt', 'Jordan', 'Kenya', 'Morocco','Saudi Arabia','The Republic of South Africa','Tunisia', 'U.A.E', 'Nigeria', 'Turkey'], 
    'Asia':['Korea','Australia','China','India','Indonesia','Japan','Malaysia','Philippines','Singapore','Taiwan','Thailand','Vietnam'], 
    'NorthAmerica' : ['Canada','Mexico','United States'],    'SouthAmerica' :['Argentina','Brazil','Chile','Colombia','Panama','Peru']
    
}
hemisphere = {
    'Northern': ['EUs', 'Austria', 'Czech Republic', 'France', 'Germany', 'Greece', 'Hungary', 'Italy', 'Netherlands', 'Poland', 'Portugal', 'Romania', 'Spain', 'Sweden', 'United Kingdom', 'Kazakhstan', 'Russia', 'Ukraine', 'Latvia', 'Israel', 'Iran', 'Jordan', 'Morocco', 'Saudi Arabia', 'Tunisia', 'Turkey', 'Korea', 'China', 'Japan', 'Taiwan', 'Canada', 'United States', 'Mexico', 'Panama'],
    'Southern': ['Algeria', 'Egypt', 'Kenya', 'The Republic of South Africa', 'U.A.E', 'Nigeria', 'Australia', 'India', 'Indonesia', 'Malaysia', 'Philippines', 'Singapore', 'Thailand', 'Vietnam', 'Argentina', 'Brazil', 'Chile', 'Colombia', 'Peru']
}
mapping_dict = {
#     "Toi muon tim hieu thong tin ky thuat, gia ca cua sp de su dung": "Product Information",
#     "tôi cần tham khảo giá và giải pháp từ LG": "Quotation or Purchase Consultation",
#     "Vui lòng báo giá giúp mình sản phẩm đo thân nhiệt Xin cảm ơn": "Request for quotation or purchase",
#     "LED Signage": "Product Information",
#     "Standalone": "Product Information",
#     "for school": "Other",
#     "Not specified": "Other",
#     "Intégrateur historique du George V": "Other",
#     "Solicito apoyo para realizar cotizacion de los dispositivos que ofrecen en la solución One Quick:": "Quotation or Purchase Consultation",
#     "Pantallas Interactivas para Clinicas": "Product Information",
#     "Hotel TV products": "Product Information",
#     "VRF": "Product Information",
#     "Preciso de um monitor médico para radiografia convencional e tomogrtafia.": "Sales Inquiry",
    "others": "Other",
    "Others": "Other",
    "other_": "Other",
    "other": "Other",
    "Etc.": "ETC.",
#     "window facing product": "Product Information",
#     "Digital platform": "Product Information",
#     "(Select ID_Needs)": "Other",
#     "One Quick:Flex": "Product Information",
#     "AIO": "Product Information",
#     "Needs": "Other",
#     "Hospital TV": "Product Information",
#     "i want to know the details about it": "Product Information",
#     "EDUCATIONAL EQUIPMENTS": "Product Information",
#     "TV interactive": "Product Information",
#     "Hola me pueden cotizar 19 pantallas interactivas de 100 pulgadas entregadas en Guayaquil -Ecuador.": "Request for quotation or purchase",
#     "teach": "Other",
#     "Display Textbook and photos": "Usage or technical consultation",
#     "High inch 86 / 98 or 110": "Product Information",
#     "quotation_": "Request for quotation or purchase",
#     "display product": "Product Information",
#     "first Info and pricing": "Quotation or Purchase Consultation",
#     "estoy buscando para Ecuador este producto LG MAGNIT micro LED, para un cliente de 138 pulgadas, con envió marítimo.": "Sales Inquiry",
#     "Evento_SdelEstero": "Other",
#     "probeam precio": "Sales Inquiry",
#     "media inquiry": "Sales Inquiry",
#     "Video Wall": "Product Information",
}


In [ ]:
# 데이터 생성 및 전처리 함수 
def get_datas():
    train = pd.read_csv("train.csv")
    test = pd.read_csv("submission.csv").drop(['id','is_converted'], axis =1) # 테스트 데이터(제출파일의 데이터)
    train['is_converted']=np.where(train['is_converted']==True,1,0)
    return train, test 


def delete_cols(data, cols):
    data = data.drop(columns=cols)
    return data

def log_transform(data,cols):
    for col in cols :
        data[col+'log']=np.log1p(data[col]) 
    return data 


def eda_expected_timeline(df):
    
    def timeline_label(time):
    
        time = str(time).lower().replace(' ','').replace('_','').replace('/','').replace(',','').replace('~','').replace('&','').replace('-','').replace('.','')
        
        if time == 'lessthan3months':
            result = 'less than 3 months'
        elif time == '3months6months':
            result = '3 months ~ 6 months'
        elif time == '6months9months':
            result = '6 months ~ 9 months'
        elif time == '9months1year':
            result = '9 months ~ 1 year'
        elif time == 'morethanayear':
            result = 'more than a year'
        else:
            result = 'aimers_0203'
            
        return result
    
    df['expected_timeline'] = df['expected_timeline'].apply(timeline_label)
    
    return df


# inquiry type 전처리하기 
def eda_inquiry_type(df):
    df['inquiry_type']= df['inquiry_type'].map(mapping_dict).fillna(train['inquiry_type'])
    df.loc[df['inquiry_type'].str.contains('Solicito apoyo para realizar', na=False), 'inquiry_type'] = 'Quotation or Purchase Consultation'
    df['inquiry_type'] = df['inquiry_type'].str.lower()
    replacement = {'/': ' ', '-':' ', '_':' '}
    df['inquiry_type'].replace(replacement, regex=True, inplace=True)
    return df


#customer type 처리 
def customer_type(data):
    data['customer_type']=data['customer_type'].fillna('none') 
    return data

# total_area 변수로 통일
def eda_business_area(df):
    for col in ['business_area','business_subarea']:
        df[col] = df[col].str.lower()
        df[col] = df[col].str.replace(" ", "") 
        df[col] = df[col].str.replace(r'[^\w\s]', "") 
        df[col] = df[col].fillna('nan') 
    df['total_area'] = df['business_area'].astype(str) + df['business_subarea'].astype(str)
    return df 

# 새로운 국가명, 대륙 열을 만들기 
def get_nation_continent(df):
    nation_corp_reverse ={v:k for k , values in nation_corp.items() for v in values }
    df['nation']=df['response_corporate'].map(nation_corp_reverse)
    continent_nation_reverse ={v:k for k , values in continent_nation.items() for v in values }
    df['continent']=df['nation'].map(continent_nation_reverse)
#     df = df.drop('customer_country',axis=1) 
    return df 

#라벨 인코딩 
def label_encoding(series: pd.Series) -> pd.Series:
    my_dict = {}
    series = series.astype(str)
    for idx, value in enumerate(sorted(series.unique())):
        my_dict[value] = idx
    series = series.map(my_dict)
    return series

# com_reg_ver_win_rate 최빈값으로 채우기 
def com_reg_fill(train,test):
    train['com_reg_ver_win_rate'] = train['com_reg_ver_win_rate'].fillna(train['com_reg_ver_win_rate'].mode()[0])
    test['com_reg_ver_win_rate'] = test['com_reg_ver_win_rate'].fillna(train['com_reg_ver_win_rate'].mode()[0])
    return train,test

def timeline_tonumber(row):
    if row['expected_timeline'] == 'less than 3 months':
        return int(1)
    elif row['expected_timeline'] == '3 months ~ 6 months':
        return int(3)
    elif row['expected_timeline'] == '6 months ~ 9 months':
        return int(6)
    elif row['expected_timeline'] == '9 months ~ 1 year':
        return int(9)    
    elif row['expected_timeline'] =='aimer_0203':
        return np.nan
    else : 
        return int(12)
    
def create_grouped_features(train, test, group, numeric_var):
    # 범주형 특성들에 대해서 다른 수치형 데이터의 중앙값, 최대, 합을 새로운 열로 추가하기 
    train = train.copy()
    test = test.copy()
    aggs = ['median', 'max','sum']
    for agg in aggs:
        # groupby 후 aggregation
        a1 = train.groupby([group])[numeric_var].agg(agg).to_dict()
        # 새로운 feature 생성
        train[numeric_var+'_'+group+'_'+agg] = train[group].map(a1)
        test[numeric_var+'_'+group+'_'+agg] = test[group].map(a1)
    return train, test

def do_scale(train,test, scale_cols) :
    for c in scale_cols:
        min_value = train[c].min()
        max_value = train[c].max()
        train[c] = (train[c] - min_value) / (max_value - min_value)
        test[c] = (test[c] - min_value) / (max_value - min_value)
    return train,test



In [ ]:
groups = ['business_unit','customer_idx']
numeric_vars = ['historical_existing_cnt', 'lead_desc_length']
scale_cols = ['com_reg_ver_win_rate','historical_existing_cnt', 'lead_desc_length','ver_win_rate_x'] 

In [ ]:
# data 갖고오기 
train,test= get_datas() 

# 스케일링 하기 
train,test =do_scale(train,test,scale_cols)
# 범주형 데이터에 대해 수치형 데이터 통계값 추가
for group in groups:
    for numeric_var in numeric_vars:
        train, test = create_grouped_features(train, test, group, numeric_var)
        
# 전처리, 로그변환 수행하기 
columns_to_log=['com_reg_ver_win_rate','lead_desc_length']
train,test= log_transform(train,columns_to_log ),log_transform(test,columns_to_log)
train,test =eda_business_area(train),eda_business_area(test)
train,test= get_nation_continent(train),get_nation_continent(test)
train,test=eda_expected_timeline(train) ,eda_expected_timeline(test)
train,test=customer_type(train) ,customer_type(test)
train,test=eda_inquiry_type(train) ,eda_inquiry_type(test)


for col in ['customer_idx','customer_type',]:
    train[col+'count'] =train[col].map(train[col].value_counts())
    test[col+'count'] =test[col].map(train[col].value_counts())
    

train['idx_unit'] = train['customer_idx'].astype(str)+train['business_unit'].astype(str)
test['idx_unit'] = test['customer_idx'].astype(str)+test['business_unit'].astype(str)
train['idx_posi'] = train['customer_idx'].astype(str)+train['customer_position'].astype(str)
test['idx_posi'] = test['customer_idx'].astype(str)+test['customer_position'].astype(str)
train['conti_inquiry'] = train['continent'].astype(str)+train['inquiry_type'].astype(str)
test['conti_inquiry'] = test['continent'].astype(str)+test['inquiry_type'].astype(str)
# train['job_unit'] = train['business_unit'].astype(str)+train['customer_job'].astype(str)
# test['job_unit'] = test['business_unit'].astype(str)+test['customer_job'].astype(str)
#0.717 나옴 4개 변수 

In [ ]:
# country.1지우지말기 
columns_to_delete=['customer_country']
# columns_to_delete=[]
train,test =delete_cols(train, columns_to_delete), delete_cols(test,columns_to_delete)

cols = [     'customer_country',    "business_subarea",    "business_area",    "business_unit",    "customer_type",    "enterprise",    "customer_job",    "inquiry_type",    "product_category",    "product_subcategory",    "product_modelname",    "customer_position",
      'customer_country.1', "response_corporate", "expected_timeline",
'nation','continent','lead_owner','idx_posi', 'conti_inquiry', 'idx_unit','bant_submit'
,'total_area'   ]
label_columns =list(set(cols)-set(columns_to_delete))

from category_encoders import CatBoostEncoder
enc = CatBoostEncoder(cols=label_columns)
enc.fit(train[label_columns], train['is_converted'])  # 'target'은 실제 데이터의 타겟 변수 이름에 맞게 변경
# 인코딩 적용
train[label_columns] = enc.transform(train[label_columns])
test[label_columns] = enc.transform(test[label_columns])

In [ ]:
test = test.fillna(0)
train = train.fillna(0)
x = train.drop('is_converted', axis=1)
y = train.is_converted

# Model

In [ ]:
# sota : random_state = 42

# dtc = DecisionTreeClassifier(random_state=i) 
# dtc.fit(x,y)

# xgb = XGBClassifier(random_state=i)
# xgb.fit(x,y)

# lgb = LGBMClassifier(random_state=i)
# lgb.fit(x,y)

# cat = CatBoostClassifier(random_state=i)
# cat.fit(x,y)

# preds1 = dtc.predict_proba(test)[:,1]
# preds2 = xgb.predict_proba(test)[:,1]
# preds3 = lgb.predict_proba(test)[:,1]
# preds4 = cat.predict_proba(test)[:,1]

# preds1 = np.array(preds1).reshape(1,-1)
# preds2 = np.array(preds2).reshape(1,-1)
# preds3 = np.array(preds3).reshape(1,-1)
# preds4 = np.array(preds4).reshape(1,-1)

# all_preds = np.concatenate((preds1,preds2),axis=0)
# all_preds = np.concatenate((all_preds,preds3),axis=0)
# all_preds = np.concatenate((all_preds,preds4),axis=0)

# final_preds = np.sum(all_preds,axis=0)

# all_preds = np.concatenate((preds1,preds2),axis=0)
# all_preds = np.concatenate((all_preds,preds3),axis=0)
# all_preds = np.concatenate((all_preds,preds4),axis=0)

# final_preds = np.sum(all_preds,axis=0)

# final_prediction = np.where(final_preds >= 0.02, 1, 0)
# np.sum(final_prediction)

In [ ]:
ls = []

for i in range(1,11):
    dtc = DecisionTreeClassifier(random_state=i) 
    dtc.fit(x,y)

    xgb = XGBClassifier(random_state=i)
    xgb.fit(x,y)

    lgb = LGBMClassifier(random_state=i)
    lgb.fit(x,y)

    cat = CatBoostClassifier(random_state=i)
    cat.fit(x,y)

    preds1 = dtc.predict_proba(test)[:,1]
    preds2 = xgb.predict_proba(test)[:,1]
    preds3 = lgb.predict_proba(test)[:,1]
    preds4 = cat.predict_proba(test)[:,1]

    preds1 = np.array(preds1).reshape(1,-1)
    preds2 = np.array(preds2).reshape(1,-1)
    preds3 = np.array(preds3).reshape(1,-1)
    preds4 = np.array(preds4).reshape(1,-1)
    
    all_preds = np.concatenate((preds1,preds2),axis=0)
    all_preds = np.concatenate((all_preds,preds3),axis=0)
    all_preds = np.concatenate((all_preds,preds4),axis=0)

    final_preds = np.sum(all_preds,axis=0)
    
    ls.append(final_preds)    

In [ ]:
vstacked_array = np.vstack(np.array(ls))

seed_preds = np.mean(vstacked_array,axis=0)

In [ ]:
vstacked_array.shape

In [ ]:
final_prediction = np.where(seed_preds >= 0.02, 1, 0)
np.sum(final_prediction)

In [ ]:
plt.hist(final_preds)

In [ ]:
# seed(3)에 대해서만 제출 할 때 
sub=pd.read_csv('submission.csv')
sub['is_converted']= final_prediction
sub.to_csv('submission.csv',index= False)
sub.to_csv('0224_nofe2_proba002_seed_ensemble.csv',index=False)